In [ ]:
import random
import pandas as pd
#Amount of Rainfall
with open('D:\monsoon.txt','r') as f:
    rain = f.readlines()
    rain = map(lambda s: s.strip(), rain)
rain = [float(i) for i in rain]
days=[30,60,40,30] # the 4 periods present in Dharnai
#Land size
acre_1 = 200 
acre_2 = 200
pump_efficiency = 0.75
motor_efficiency = 0.75
land_1_load = []
land_2_load = []
wheat_mm = []
rice_1_mm = []
rice_2_mm = []
total = []
rice_3_mm = []
sugarcane_mm = []
tomato_mm = []
groundnut_mm = []
land_1_mm = []
land_2_mm = []
rainpd = 0
rain_per_day = [0]*153
r = 0
load = []
#The water requirements for various days of various seasons are considered based on the below code for the four crops
for day in range(365):     # Wheat 
    if day>=325 and day<355:
        wheat_mm.append(random.uniform(9.5,10.5))
    elif day>=355 or day<50:
        wheat_mm.append(random.uniform(5.75,6.75))
    elif day>=50 and day<90:
        wheat_mm.append(random.uniform(7.5,8.75))
    elif day>=90 and day<=120:
        wheat_mm.append(random.uniform(5.75,6.75))
    else:
        wheat_mm.append(0)
for day in range(365):              # Rice_1
	if day>=135 and day<165:
		rice_1_mm.append(random.uniform(4.25,5.75))
	elif day>=165 and day<225:
		rice_1_mm.append(random.uniform(6.5,7.75))
	elif day>=225 and day<265:
		rice_1_mm.append(random.uniform(10,12.5))
	elif day>=265 and day<=295:
		rice_1_mm.append(random.uniform(5,6.5))
	else:
		rice_1_mm.append(0)
for day in range(365):          #Rice_2
	if day>=195 and day<225:
		rice_2_mm.append(random.uniform(4.25,5.75))
	elif day>=225 and day<285:
		rice_2_mm.append(random.uniform(6.5,7.75))
	elif day>=285 and day<325:
		rice_2_mm.append(random.uniform(10,12.5))
	elif day>=325 and day<=355:
		rice_2_mm.append(random.uniform(5,6.5))
	else:
		rice_2_mm.append(0)
for day in range(365):        # Rice_3
	if day>=15 and day<45:
		rice_3_mm.append(random.uniform(4.25,5.75))
	elif day>=45 and day<105:
		rice_3_mm.append(random.uniform(6.5,7.75))
	elif day>=105 and day<145:
		rice_3_mm.append(random.uniform(10,12.5))
	elif day>=145 and day<=175:
		rice_3_mm.append(random.uniform(5,6.5))
	else:
		rice_3_mm.append(0)
#TO cosnider Rainfall
for n in range(len(rain))        # Rainfall
	if rain[n]==1:
		rain[n]=random.uniform(0.05,0.25)
	elif rain[n]==2:
		rain[n]=random.uniform(0.25,1)
	elif rain[n]==3:
		rain[n]=random.uniform(1,4)
	elif rain[n]==4:
		rain[n]=random.uniform(4,16)
for n in range(len(rain)):                    
    if n%24==0 and n>0:
        rain_per_day.append(0.75*rainpd)
        rainpd=0
    if n==len(rain)-1:
        rain_per_day.append(0.75*rainpd)
    rainpd=rainpd+rain[n]
for n in range(90):
	rain_per_day.append(0)

for day in range(365):        # MM without rainfall
	land_1_mm.append(rice_1_mm[day]+wheat_mm[day])
	land_2_mm.append(rice_2_mm[day]+rice_3_mm[day])

for day in range(365):         # Addition of rainfall
	if day>152 and day<275:
		r=r+rain_per_day[day]
		if r>=land_1_mm[day]:
			r=r-land_1_mm[day]
			land_1_mm[day]=0
		else:
			land_1_mm[day]=land_1_mm[day]-r
			r=0
r = 0
for day in range(365):
	if day>152 and day<275:
		r = r+rain_per_day[day]
		if r>=land_2_mm[day]:
			r=r-land_2_mm[day]
			land_2_mm[day]=0
		else:
			land_2_mm[day]=land_2_mm[day]-r
			r=0
gpm_hp = 3960 #Conversion factor for gpm per foot to hp
hp_kW = 0.746 #Conversion factor for horsepower to kW
for day in range(365):            # Land 1
    land_1_gpm=(0.74*land_1_mm[day]*acre_1)/(random.uniform(0.65,0.75)*6) # Daily Variation (pump operates for 6 hours a day)
    for hour in range((day*24),(day*24)+24):
        if hour%24<16 and hour%24>=10:
            pump_efficiency=random.uniform(0.6,0.75)       
            motor_efficiency=random.uniform(0.65,0.80)
            land_1_load.append((land_1_gpm*50*hp_kW)/(gpm_hp*motor_efficiency*pump_efficiency))
        else:
            land_1_load.append(0)
for day in range(365):         # Land 2
    land_2_gpm=(0.74*land_2_mm[day]*acre_2)/(random.uniform(0.65,0.75)*6) 
    for hour in range((day*24),(day*24)+24):
        if hour%24<16 and hour%24>=10:
            pump_efficiency=random.uniform(0.6,0.75)  
            motor_efficiency=random.uniform(0.65,0.80)
            land_2_load.append((land_2_gpm*50*hp_kW)/(gpm_hp*motor_efficiency*pump_efficiency))
        else:
            land_2_load.append(0)
for hour in range(8760):
    load.append(land_1_load[hour]+land_2_load[hour])
    total.append(land_1_load[hour]+land_2_load[hour])
df = pd.DataFrame()
for i in range(365*24):
    row = [load[i],i%24]
    df1 = pd.DataFrame(row)
    df1 = df1.T
    df = df.append(df1)
df.columns=(["load","hours"])
changedload = []
#Modeeling the load for 20 years (Average increase of around .1 is considered per year)
for year in range(20):
    data = [load]
    df=pd.DataFrame(data)
    df=df.T
    changedload=load
    load=[]
    df.to_csv('irr6hrload_'+ str(year) +'.csv',index=False,encoding='utf-8')
    for hour in range(8760):
        load.append(changedload[hour]+random.uniform(0.05,0.15)*changedload[hour])
        total.append(load[hour])
final_load = pd.DataFrame()
data = [(i,total[i]) for i in range (0,365*24*20)]
final_load = pd.DataFrame(data,columns=['hour','load'])